In [ ]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
from dask.dot import dot_graph
import itertools
import logging
import netCDF4
import numpy as np
import dask.array as da
from dask import delayed
import time
from dask.distributed import Client
from urllib import request
from multiprocessing import Pool

client = Client('scheduler:8786')

download_location = '/temp'
# data_url = 'http://nasanex.s3.amazonaws.com'
data_url = 'http://192.168.1.123:8080'
max_download_attempts = 5

all_models = ['ACCESS1-0',  'BNU-ESM', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'CanESM2', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM-CHEM', 'MIROC-ESM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M', 'bcc-csm1-1', 'inmcm4']
# all_models = ['ACCESS1-0', 'BNU-ESM'] 
all_vars = ['tasmax', 'pr']
all_years = {
     # 'historical': list(range(1971, 1976))
     # 'historical': list(range(1971, 2001))
    'historical': [1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000]
}

def get_dataset_url(variable, scenario, model, year, prefix = data_url):
    prefix_filename = '/'.join([prefix, 'NEX-GDDP', 'BCSD', scenario, 'day', 'atmos', variable, 'r1i1p1', 'v1.0'])
    filename = '_'.join([variable, 'day', 'BCSD', scenario, 'r1i1p1', model, str(year) + '.nc'])
    url = '/'.join([prefix_filename, filename])
    return url

def get_context(year, **kwargs):
    variables = [kwargs.get('variable')] if kwargs.get('variable') else all_vars
    scenarios = ['historical']
    models = [kwargs.get('model')] if kwargs.get('model') else all_models
    outlist = []
    combinations = list(itertools.product(variables, scenarios, models))
    result = list(map(lambda comb: [ *comb, year ], combinations))
    return result

def get_year_ensemble(year, variable = 'tasmax'):
    context = get_context(year, variable = variable)
    datasets = list(map(lambda x: str(get_dataset_url(*x)), context))
    return datasets

def download_file(url):
    print("url: " + url)
    attempts = 0
    success = False
    filename = ""
    while attempts < max_download_attempts and not success:
        time.sleep(2 ** attempts)
        filename = '/'.join([download_location, str(url.split('/')[-1])])
        print("Downloading file at " + filename)
        u = request.urlopen(url)
        f = open(filename, 'wb')
        f.write(u.read())
        f.close()
        success = True
        break
    return filename

def download_file_list(url_list):
    print("Starting download pool")
    pool = Pool()
    res = pool.map(download_file, url_list)
    print("Jobs sent")
    pool.close()
    pool.join()
    print("Downloads finished")
    print(res)
    return res

def download_and_stack(year, variable):
    dsets_urls = list(map(lambda x: get_year_ensemble(x, variable = variable), [year]))[0]
    filenames = download_file_list(dsets_urls)
    datasets = [ netCDF4.Dataset(filename) for filename in filenames ]
    dask_arrays = []
    for dset in datasets:
        dask_array = da.from_array(dset[variable], chunks = (366, 144, 144))
        if dask_array.shape == (366, 720, 1440):
            dask_array_noleap = dask_array[[*list(range(0, 60)), *list(range(61, 366))],:,:]
            print(dask_array_noleap)
            dask_arrays.append(dask_array_noleap)
        else:
            print(dask_array)
            dask_arrays.append(dask_array)
    try:
        final_stack = da.stack(dask_arrays, axis = 0)
    except:
        return filenames, None, None
    return filenames, datasets, final_stack

def get_stacks_mod_avg(a, chunksize):
    nmodels, time, lat, lon = a.shape
    nstacks_lat = int(np.ceil(lat / chunksize))
    nstacks_lon = int(np.ceil(lon / chunksize))
    
    stacks = []
    
    for i in range(nstacks_lat):
        for j in range(nstacks_lon):
            latmin, latmax = i * chunksize, (i+1) * chunksize
            lonmin, lonmax = j * chunksize, (j+1) * chunksize
            print(i, j, '~>', latmin, latmax, lonmin, lonmax)
            stacked = a[:, :, latmin:latmax, lonmin:lonmax]
            print(stacked)
            stacks.append(stacked)
    return stacks

def copy_dataset(src, output_filename):
    dst = netCDF4.Dataset('/home/jovyan/work/' + output_filename, "w")
    # copy attributes
    for name in src.ncattrs():
        dst.setncattr(name, src.getncattr(name))
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(
            name, (len(dimension) if not dimension.isunlimited else None))
    dst.close()
    return output_filename

def restack(chunk_list, chunksize):
    shapes = list(map(np.shape, chunk_list))
    ndays = shapes[0][0]
    nlons = int(1440 / chunksize)
    nlats = int(720 / chunksize)

    out_array = np.empty((ndays, 720, 1440))
 
    combs = list(itertools.product(
        list(range(nlats)),
        list(range(nlons))
    ))
    
    res_list = zip(combs, chunk_list)
    
    for position, arr in res_list:
        minlon, maxlon = position[0] * chunksize, position[0] * chunksize + chunksize
        minlat, maxlat = position[1] * chunksize, position[1] * chunksize + chunksize
        out_array[:, minlon:maxlon, minlat:maxlat] = arr
    return out_array

In [ ]:
client.restart()

In [ ]:
# Processing pr years
years = list(range(1971, 2001))
years = [1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000]
var = 'pr'
for i, year in enumerate(years):
    filenames, datasets, current_year_stack = download_and_stack(year, variable=var)
    if current_year_stack is not None:
        chunked_stack = get_stacks_mod_avg(current_year_stack, 360)
        mean = list(map(lambda x: delayed(np.mean)(x, axis=0).compute(), chunked_stack))
        out_array = restack(mean, 360)
        np.save('/home/jovyan/work/' + str(year) + '_pr_avg.npy', out_array)
        del chunked_stack
        del mean
        del out_array
        del current_year_stack
        #output_dataset_filename = copy_dataset(datasets[0], str(year) + '_ensemble_' + var + '.nc')
        #output_dataset = netCDF4.Dataset(output_dataset_filename, 'w')
        #print(output_dataset)
        
    # Finally
    for filename in filenames:
        os.remove(filename)

In [ ]:
ls -l /temp/

In [ ]:
def download_and_stack(year, variable):
    dsets_urls = list(map(lambda x: get_year_ensemble(x, variable = variable), [year]))[0]
    filenames = download_file_list(dsets_urls)
    datasets = [ netCDF4.Dataset(filename) for filename in filenames ]
    dask_arrays = []
    for dset in datasets:
        dask_array = da.from_array(dset[variable], chunks = (366, 144, 144))
        print(dask_array)
        if dask_array.shape == (366, 720, 1440):
            dask_array_noleap = dask_array[[*list(range(0, 60)), *list(range(61, 366))],:,:]
            print(dask_array_noleap)
            dask_arrays.append(dask_array_noleap)
        else:
            dask_arrays.append(dask_array)
    try:
        final_stack = da.stack(dask_arrays, axis = 0)
    except:
        return filenames, None, None
    return filenames, datasets, final_stack

download_and_stack(1972, 'pr')

In [ ]:
rm /temp/*

In [ ]:
ls /temp/

In [2]:
import glob
import numpy as np
year_matrixes = sorted([fname for fname in glob.glob('/temp/*.npy')])
year_matrixes


['/temp/1971_pr_avg.npy',
 '/temp/1972_pr_avg.npy',
 '/temp/1973_pr_avg.npy',
 '/temp/1974_pr_avg.npy',
 '/temp/1975_pr_avg.npy',
 '/temp/1976_pr_avg.npy',
 '/temp/1977_pr_avg.npy',
 '/temp/1978_pr_avg.npy',
 '/temp/1979_pr_avg.npy',
 '/temp/1980_pr_avg.npy',
 '/temp/1981_pr_avg.npy',
 '/temp/1982_pr_avg.npy',
 '/temp/1983_pr_avg.npy',
 '/temp/1984_pr_avg.npy',
 '/temp/1985_pr_avg.npy',
 '/temp/1986_pr_avg.npy',
 '/temp/1987_pr_avg.npy',
 '/temp/1988_pr_avg.npy',
 '/temp/1989_pr_avg.npy',
 '/temp/1990_pr_avg.npy',
 '/temp/1991_pr_avg.npy',
 '/temp/1992_pr_avg.npy',
 '/temp/1993_pr_avg.npy',
 '/temp/1994_pr_avg.npy',
 '/temp/1995_pr_avg.npy',
 '/temp/1996_pr_avg.npy',
 '/temp/1997_pr_avg.npy',
 '/temp/1998_pr_avg.npy',
 '/temp/1999_pr_avg.npy',
 '/temp/2000_pr_avg.npy']

In [ ]:
res = np.load('/temp/1997_pr_avg.npy')
res.shape

In [ ]:
def slice_constructor(shape, axis, chunksize):
    out = []
    shape_list = list(shape)
    del shape_list[axis]
    
    def slicer(*args):
        slices = []
        for arg, dim in zip(args, shape_list):
            slices.append(slice(arg * chunksize, (arg + 1) * chunksize))
        slices.insert(axis, slice(0, shape[axis] ))
        return slices
    return slicer

def apply_by_chunks(f, axis, arrs, arrsize, chunksize, **kwargs):
    slicer = slice_constructor(arrsize, axis, chunksize)
    total_results = []
    for lat in range(0, 10):
        for lon in range(0, 20):
            print("processing " + str(lat) + " " + str(lon))
            slice = slicer(lat, lon)
            print(slice)
            
            current_processing_stack = []
            for arr_filename in arrs:
                print("Processing " + arr_filename)
                original_matrix = np.load(arr_filename)
                copied_slice = np.array(original_matrix[slice], copy=True)
                del original_matrix
                current_processing_stack.append(copied_slice)
                # print(current_processing_stack)
                # print(current_processing_stack.shape)
            print("Stacking")
            final_stack = np.concatenate(current_processing_stack, axis = axis)
            np.save('/temp/' + str(lat) + '_' + str(lon), final_stack)
            print("Calculating")
            result = f(final_stack, axis = axis, **kwargs)
            np.save('/temp/result_' + str(lat) + '_' + str(lon), result)
            print(result)
            print(result.shape)
            total_results.append(result)
            
            
            
    

apply_by_chunks(np.percentile, 0, year_matrixes, (365, 720, 1440), 72, q=99)
# f = slice_constructor((365, 720, 1440), 0, 72)
# [slice(0, 365, None), slice(360, 432, None), slice(504, 576, None)]
# f(5, 7)

processing 0 0
[slice(0, 365, None), slice(0, 72, None), slice(0, 72, None)]
Processing /temp/1971_pr_avg.npy
Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy

Calculating
[[  7.14285854e+19   7.14285854e+19   7.14285854e+19 ...,   7.14285854e+19
    7.14285854e+19   7.14285854e+19]
 [  7.14285854e+19   7.14285854e+19   7.14285854e+19 ...,   7.14285854e+19
    7.14285854e+19   7.14285854e+19]
 [  6.19047696e+19   6.19047696e+19   6.19047696e+19 ...,   6.19047696e+19
    6.19047696e+19   6.19047696e+19]
 ..., 
 [  1.63417037e-05   1.63012613e-05   1.62510345e-05 ...,   2.11376447e-05
    2.10784405e-05   2.12748815e-05]
 [  1.64578288e-05   1.64218243e-05   1.66414575e-05 ...,   2.09774735e-05
    2.10265599e-05   2.10972697e-05]
 [  1.69241218e-05   1.68786546e-05   1.70821067e-05 ...,   2.09838789e-05
    2.09851796e-05   2.11326285e-05]]
(72, 72)
processing 0 5
[slice(0, 365, None), slice(0, 72, None), slice(360, 432, None)]
Processing /temp/1971_pr_avg.npy
Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977

Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  7.14285854e+19   7.14285854e+19   7.14285854e+19 ...,   7.14285854e+19
    7.14285854e+19   7.14285854e+19]
 [  7.14285854e+19   7.14285854e+19   7.14285854e+19 ...,   7.14285854e+19
    7.14285854e+19   7.14285854e+19]
 [  6.19047696e+19   6.19047696e+19   6.19047696e+19 ...,   6.

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  7.14285854e+19   7

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  7.14285854e+19   7

...

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  5.35941637e-05   5

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  6.97920538e-05   6

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  7.29573896e-05   7

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
Processing /temp/1979_pr_avg.npy
Processing /temp/1980_pr_avg.npy
Processing /temp/1981_pr_avg.npy
Processing /temp/1982_pr_avg.npy
Processing /temp/1983_pr_avg.npy
Processing /temp/1984_pr_avg.npy
Processing /temp/1985_pr_avg.npy
Processing /temp/1986_pr_avg.npy
Processing /temp/1987_pr_avg.npy
Processing /temp/1988_pr_avg.npy
Processing /temp/1989_pr_avg.npy
Processing /temp/1990_pr_avg.npy
Processing /temp/1991_pr_avg.npy
Processing /temp/1992_pr_avg.npy
Processing /temp/1993_pr_avg.npy
Processing /temp/1994_pr_avg.npy
Processing /temp/1995_pr_avg.npy
Processing /temp/1996_pr_avg.npy
Processing /temp/1997_pr_avg.npy
Processing /temp/1998_pr_avg.npy
Processing /temp/1999_pr_avg.npy
Processing /temp/2000_pr_avg.npy
Stacking
Calculating
[[  6.57727831e-05   6

Processing /temp/1972_pr_avg.npy
Processing /temp/1973_pr_avg.npy
Processing /temp/1974_pr_avg.npy
Processing /temp/1975_pr_avg.npy
Processing /temp/1976_pr_avg.npy
Processing /temp/1977_pr_avg.npy
Processing /temp/1978_pr_avg.npy
